# Modelling module
## HYDROTEL
The `xhydro.modelling.Hydrotel` class is used to prepare and update configuration files for HYDROTEL, validate the project directory and the meteorological inputs, execute the model, and reformat the outputs to be more inline with CF conventions and other functionalities of `xhydro`. Note that it does not prepare the project directory itself, which should be done beforehand.

<div class="alert alert-info"> <b>INFO</b>
`xhydro` does not distribute the HYDROTEL executable. If you want to use this module, you will have to get access to it first.
</div>

### Initialising the model
A new call to HYDROTEL will typically start by modifying some entries in the three main configuration files (`projet.csv`, `simulation.csv`, and `output.csv`). When initialising a new Hydrotel class, you can either read the configuration from the files that already exist, or use default configuration options defined in `xhydro/modelling/data/hydrotel_defaults.yml`.

The arguments of the class are:

- `project`: *str* or *os.Pathlike*. The full path to the project directory.
- `default_options`: *bool*. Whether to use the aforementioned default configuration options, or read them from the files on disk.
- `project_options`, `simulation_options`, `output_options`: *dict*. Configuration options to overwrite with new information.

In [ ]:
from pathlib import Path
from xhydro.modelling import Hydrotel
import xhydro as xh

# Folder where to put the data
project_folder = Path().absolute() / "_data"
project_name = "example_hydrotel"

In [ ]:
# This is a hidden cell. We'll create a fake Hydrotel directory for the purpose of this example.
from xclim.testing.helpers import test_timeseries as timeseries
import xhydro.testing.utils
import numpy as np

# Fake meteorological data
meteo = timeseries(
    np.zeros(365 * 2),
    start="2001-01-01",
    freq="D",
    variable="tasmin",
    as_dataset=True,
    units="degC",
)
meteo["tasmax"] = timeseries(
    np.ones(365 * 2),
    start="2001-01-01",
    freq="D",
    variable="tasmax",
    units="degC",
)
meteo["pr"] = timeseries(
    np.ones(365 * 2) * 10,
    start="2001-01-01",
    freq="D",
    variable="pr",
    units="mm",
)
meteo = meteo.expand_dims("stations").assign_coords(stations=["010101"])
meteo = meteo.assign_coords(
    coords={"lat": 46, "lon": -77, "x": 0, "y": 0, "z": 0}
)
for c in ["lat", "lon", "x", "y", "z"]:
    meteo[c] = meteo[c].expand_dims("stations")

# Fake output
debit_aval = timeseries(
    np.zeros(365 * 2),
    start="2001-01-01",
    freq="D",
    variable="streamflow",
    as_dataset=True,
)
debit_aval = debit_aval.expand_dims("troncon").assign_coords(troncon=[0])
debit_aval = debit_aval.assign_coords(coords={"idtroncon": 0})
debit_aval["idtroncon"] = debit_aval["idtroncon"].expand_dims("troncon")
debit_aval = debit_aval.rename({"streamflow": "debit_aval"})
debit_aval["debit_aval"].attrs = {
    "units": "m3/s",
    "description": "Debit en aval du troncon",
}

xhydro.testing.utils.fake_hydrotel_project(
    project_folder, project_name, meteo=meteo, debit_aval=debit_aval
)

In [ ]:
# The options only need to be those that differ from the defaults/on-disk. These will generally include the simulation dates and output requested.
simulation_options = {"DATE DEBUT": "2001-01-01", "DATE FIN": "2001-12-31", "FICHIER STATIONS METEO": "meteo/SLNO_meteo_GC3H.nc", "PAS DE TEMPS": 24}
output_options = {"TRONCONS": 1, "DEBITS_AVAL": 1}
ht = Hydrotel(project_folder / project_name, default_options=False, simulation_options=simulation_options, output_options=output_options)

print(f"Simulation name, taken from 'projet.csv': '{ht.simulation_name}'\n")
print(f"Project configuration: '{ht.project_options}'\n")
print(f"Simulation configuration: '{ht.simulation_options}'\n")
print(f"Output configuration: '{ht.output_options}'")

## Validating the meteorological data
A few basic checks will be automatically performed prior to executing the model, but a user might want to perform more advanced health checks (missing values, unrealistic meteorological inputs, etc.). This is possible through the use of `xhydro.utils.health_checks`. Consult [the 'xscen' documentation](https://xscen.readthedocs.io/en/latest/notebooks/3_diagnostics.html#Health-checks) for the full list of possible checks.

Here, we'll:
- Make sure that there is no missing data.
- That there are no abnormal meteorological values or sequence of values. Some checks here will fail, due to the nature of our fake data.

In [ ]:
health_checks = {"raise_on": [],  # If an antry is not here, it will warn the user instead of raising an exception.
                 "missing": {"missing_any": {"freq": "YS"}},
                 "flags": {
                     "pr":  # You can have specific flags per variable.
                           {  
                               "negative_accumulation_values": {},
                               "very_large_precipitation_events": {},
                               "outside_n_standard_deviations_of_climatology": {"n": 5},
                               "values_repeating_for_n_or_more_days": {"n": 5}
                               },
                           "tasmax": 
                           {
                               "tasmax_below_tasmin": {},
                               "temperature_extremely_low": {},
                               "temperature_extremely_high": {},
                               "outside_n_standard_deviations_of_climatology": {"n": 5},
                               "values_repeating_for_n_or_more_days": {"n": 5},
                               },
                           "tasmin": 
                           {
                               "temperature_extremely_low": {},
                               "temperature_extremely_high": {},
                               "outside_n_standard_deviations_of_climatology": {"n": 5},
                               "values_repeating_for_n_or_more_days": {"n": 5},
                               },
                     }
                 }

In [ ]:
ds_in = ht.get_input()
ds_in["pr"].attrs["units"] = "mm d-1"  # Hydrotel-to-xclim compatibility

xh.utils.health_checks(ds_in, **health_checks)

## Executing HYDROTEL
A few basic checkups will always be performed prior to executing HYDROTEL. `HYDROTEL.run()` will always make sure that:

- All files mentioned in the configuration exist.
- The dataset has both "time" and "stations" dimensions.
- The dataset has "lat", "lon", "x", "y", "z" coordinates.
- The dataset has "tasmin" (degC), "tasmax" (degC), and "pr" (mm) variables.
- The dataset has a standard calendar.
- The frequency is uniform (i.e. all time steps are equally spaced).
- The start and end dates are contained in the dataset.

Only when those checks are satisfied will the function actually execute the model. In addition, the following arguments can be called:

- `hydrotel_console`: *str* or *os.Pathlike*. For Windows, this is the path to the Hydrotel executable.
- `id_as_dim`: *bool*. By default, Hydrotel will generate a dataset with `troncon` as the spatial dimension, and `idtroncon` as a coordinate. If True, this will swap them so that `idtroncon` is the dimension, then will remove `troncon` altogether.
- `dry_run`: *bool*. True by default, just to prevent erroneous executions. Put at False to execute the model.

Once HYDROTEL has been run, `xhydro` will automatically reformat the NetCDF to bring it closer to CF conventions and into a format that can be used with other `xhydro` modules, although no standard exists for hydrological data. This currently only supports the "DEBITS_AVAL" output option.

In [ ]:
# This is what raw HYDROTEL outputs look like:
ht.get_streamflow()  # Use this function to automatically retrieve the generated streamflow. We used a fictitious example, which is why the file already exists.

In [ ]:
# For the purpose of this example, we'll leave 'dry_run' as True.
print("Command that would be run in the terminal:")
ht.run(id_as_dim=True, dry_run=True)

In [ ]:
# This cell is hidden.
ht._standardise_outputs()

If we examine the streamflow file after executing HYDROTEL, we'll see that it has been reformatted.

- `troncon` and `idtroncon` are renamed to `station` and `station_id`, respectively.
- If `id_as_dim` is True, then `station_id` is the dimension.
- `debit_aval` is renamed `streamflow` and is given standard attributes.

In [ ]:
ds = ht.get_streamflow()
ds

In [ ]:
# This cell is hidden.
import os
import shutil

if os.path.isdir(project_folder):
    shutil.rmtree(project_folder)